In [ ]:
import pandas as pd #data manipulation package to import for analysis
import numpy as np #package for multidimensional array computations
import matplotlib.pyplot as plt #library for plotting (matlab)
import datetime #supplies classes for manipulating date and time
pip install getFamaFrenchFactors

#the following lines get the last 5 years of monthly stock price data for a given stock (Macy's) and sets it into a dataFrame

macyData=DataReader('M','yahoo','2016-07-01',end)['Adj Close'] #scrapes data from yahoo finance
macyDataF=macydata.resample('M').last() #takes only last day of the month
macyDataF=macydataF.to_frame() #puts it into the data frame
macyDataF['Year']=macyDataF.index.year #takes year data
macyDataF['Month']=macyDataF.index.month #takes month data
macyDataF.rename(columns={'Adj Close':'M'},inplace=True) #sets adjusted close column

#the following lines get the past 5 years of data from the fama french library

FF3data=gff.famaFrench3Factor(frequency='m') #gets monthly data from the library
FF3data=FF3data.tail(60) #takes last 60 entries
FF3data['Year']=FF3data['date_ff_factors'].dt.year #sets year column
FF3data['Month']=FF3data['date_ff_factors'].dt.month #sets month column

#the following lines merge the data into a single table

datanow=pd.merge(macyDataF,FF3data,left_on=['Year','Month'],right_on=['Year','Month']) #merges on month and year 

datanow.sort_values(['date_ff_factors'],axis=0,ascending=False,inplace=True) #sorts by date
datanow['rM']=datanow['M']/datanow.M.shift(-1)-1 #dividing by yesterdays (minus 1)'s price

#following lines clean up and set together tables
datanow.dropna(subset=['rM'],inplace=True)
datanow['rMkt']=datanow['Mkt-RF']+datanow['RF']

#scatter plot of the market vs the security
plt.scatter(datanow['rMkt'],datanow['M'])
m, b = np.polyfit(datanow['rMkt'],datanow['rM'],1)
plt.plot(datanow['rMkt'],b+m*datanow['rMkt'],'r')

import statsmodels.api as sm #imports statistical models for analysis
y=datanow['rM']-datanow['RF'] #sets the y variable
x=datanow['Mkt-RF'] #sets x variable
x=sm.add_constant(x)

#creates model parameters
mod=sm.OLS(y,x)
res=mod.fit()
res.summary()



